# **Pneumonia Detection from X-rays**
## Project Documentation
#### Authors: [Sebastian Fredin, Ali Hussain, Julia Sjöholm, Younis Moussavi]
#### Course: Deep-Learning, AI23 IT-Högskolan Göteborg

# 1. Project Overview
This project implements a deep learning solution for classifying chest X-ray images into:
- Healthy vs Pneumonia (binary classification)
- Normal vs Bacterial vs Viral Pneumonia (multi-class classification)

The project uses convolutional neural networks (CNNs) to analyze X-ray images and provide accurate pneumonia detection.

# 2. Data Analysis and Exploration
From the data exploration notebook (data_exploration.ipynb), we can see important dataset characteristics:

### Dataset Distribution:
- Training set:
  - Normal: 1341 images
  - Bacterial: 2530 images
  - Viral: 1345 images
- Test set:
  - Normal: 234 images
  - Bacterial: 242 images
  - Viral: [number] images

### Image Characteristics:
- Most images are grayscale (5573 out of 5856)
- Mean pixel intensity: 122.79
- Standard deviation: 18.39
- Only 2 anomalous images identified with unusual brightness levels

# 3. Project Structure
The project follows a well-organized structure as defined in the README.md:


PneumoScope/
│
├── data/                  # Folder for datasets
│   ├── train/             # Training images
│   ├── test/              # Testing images
│   └── val/               # Validation images (optional)
│
├── notebooks/             # Jupyter notebooks for experiments and EDA
│   ├── data_exploration.ipynb
│   └── model_training.ipynb
│
├── src/                   # Source code
│   ├── data_loader.py     # Code to load and preprocess the dataset
│   ├── model.py           # Code to define the model architecture
│   ├── train.py           # Script to train the model
│   ├── evaluate.py        # Script to evaluate the model
│   └── utils.py           # Helper functions
│
├── saved_models/          # Folder to save trained models
│   ├── best_model.h5
│   └── latest_model.h5
│
├── results/               # Folder for storing results and logs
│   ├── plots/             # Training/validation curves, confusion matrices
│   └── logs/              # Training logs
│
├── main.py                # Main script, runs training and evaluation
├── requirements.txt       # List of dependencies
├── reorganize_dataset.py  # Resizes train|val|test sizes
├── README.md              # Project description and instructions
├── .gitignore             # Ignored files/folders
└── LICENSE                # License for the project

# 4. Implementation Steps
Based on the ToDo.md file, the project was implemented in several phases:


## **2. Data Preparation**
- [X] Download the dataset from Kaggle. # Sebbe data_loader.py
- [X] Organize the dataset into appropriate directories (`train`, `test`, `val`).
- [X] Explore the dataset (e.g., check for class imbalance, image size). # Julia data_exploration.ipynb
- [X] Perform data preprocessing: 
  - [X] Resize images to a uniform size.
  - [X] Normalize image pixel values.
  - [X] Apply data augmentation (e.g., flipping, rotation, zoom).

## **3. Model Development**
- [X] Choose baseline CNN architecture (e.g., simple CNN). # Ali
- [X] Train a binary classification model (pneumonia vs. no pneumonia).
- [X] Train a ternary classification model (bacterial vs. viral vs. no pneumonia). # Ali
- [X] Experiment with pre-trained models (e.g., ResNet, VGG, Inception).
- [X] Implement transfer learning if using pre-trained models.

## **4. Model Evaluation** # Ali och Julia
- [X] Evaluate models on the test set.
- [X] Compute key metrics:
  - [X] Accuracy
  - [X] Precision, Recall, F1-score, Accuracy
  - [X] ROC-AUC # Ali
- [X] Create and display confusion matrices. # Julia
- [X] Save model performance results for comparison. # Ali

## **5. Optimization** # Sebastian
- [X] Fine-tune hyperparameters (learning rate, batch size, number of epochs). 
- [X] Implement early stopping and learning rate scheduling. # Early stopping check, learning rate scheduler not yet implemented
- [ ] Test different optimizers (e.g., Adam, SGD).
- [X] Experiment with different data augmentation strategies. # Implemented but not tested

---

# Pipeline Documentation

## main.py
This file orchestrates the complete machine learning pipeline for pneumonia detection from X-ray images, both binary and multiclass classification tasks.
### Pipeline Steps

def main():

Main pipeline:
1) **Data Download**: Download data from Kaggle if not present
2) **Data Reorganization**: Reorganize data for binary/multiclass if needed
3) **Binary Classification**: Train and evaluate binary classifier
4) **Multiclass Classification**: Train and evaluate multiclass classifier


### Key Features
1. **Automatic Data Management**
   - Downloads dataset if missing
   - Organizes data for both classification tasks

2. **Binary Classification**
   - Trains model (pneumonia vs. normal)
   - Evaluates performance metrics
   - Saves model and results

3. **Multiclass Classification**
   - Trains model (normal vs. bacterial vs. viral)
   - Evaluates performance metrics
   - Saves model and results

### Output Metrics
- Binary: Precision, Recall, F1, Accuracy, AUC
- Multiclass: Macro-averaged metrics (Precision, Recall, F1) and Accuracy

### Usage
Run `python main.py` to execute the complete pipeline

---

# Web Application 

## app.py
A Flask web application that serves a pneumonia detection model through a user interface.


### Key Features
1. **Model Setup**
   - Loads pre-trained PneumoNet model
   - Configures for CPU/GPU automatically
   - Handles image preprocessing

2. **Routes**
   - `/`: Home page with upload interface
   - `/predict`: Handles image processing and prediction

3. **Prediction Pipeline**
   - Accepts uploaded X-ray images
   - Preprocesses images to model requirements
   - Returns binary classification (Normal/Pneumonia)

### Usage
1. Run `python app.py`
2. Open browser to `http://localhost:5000`
3. Upload X-ray image for prediction

### Security Features
- Temporary file handling
- Input validation
- Automatic file cleanup

---

# Data Loading 

## data_loader.py
Handles data loading and preprocessing for the pneumonia detection model, including data augmentation and class balancing.

### Key Components
def get_data_loaders(data_dir, batch_size, num_workers, augment_train, random_crop,
color_jitter, balance_train, desired_total_samples, multiclass=False):

Creates DataLoaders for training, validation, and testing
with configurable augmentation and balancing options.

### Features
1. **Data Transforms**
   - Resizing (224x224 or 256→224 with random crop)
   - Augmentations: horizontal flip, rotation, color jitter
   - Normalization using ImageNet statistics

2. **Data Loading**
   - Separate loaders for train/val/test
   - Class verification against expected labels
   - Support for both binary and multiclass

3. **Class Balancing**
   - Optional weighted sampling for imbalanced datasets
   - Configurable total samples
   - Uses `WeightedRandomSampler`

### Parameters
- `data_dir`: Dataset root directory
- `batch_size`: Samples per batch
- `augment_train`: Enable training augmentations
- `balance_train`: Enable class balancing
- `multiclass`: Toggle binary/multiclass mode

### Returns
Three DataLoaders: training, validation, and testing

---

# Multiclass Evaluation

## evaluate_multiclass.py
Evaluates the performance of the multiclass pneumonia detection model (Normal vs. Bacterial vs. Viral).

### Key Components
def evaluate_model_multiclass(model_path, data_dir, results_dir='results/multiclass'):

Evaluates trained model on test dataset and calculates performance metrics.

### Features

1. **Model Evaluation**
   - Loads trained model from checkpoint
   - Processes test dataset
   - Calculates comprehensive metrics

2. **Performance Metrics**
   - Confusion matrix
   - Macro-averaged metrics (class-independent)
   - Micro-averaged metrics (class-size aware)
   - Per-class performance analysis

3. **Results Storage**
   - Saves metrics as JSON
   - Includes detailed per-class analysis
   - Stores confusion matrix

### Key Metrics
- Precision (macro/micro)
- Recall (macro/micro)
- F1-score (macro/micro)
- Accuracy
- Per-class performance

### Output
- Console output with detailed metrics
- JSON file with complete evaluation results
- Confusion matrix visualization

---

# Binary Evaluation

## evaluate.py
Evaluates the performance of the binary pneumonia detection model (Normal vs. Pneumonia).

### Key Components
def evaluate_model(model_path, data_dir, results_dir='results', plot_cm=False):

Evaluates trained model on test dataset and generates comprehensive metrics.


### Features
1. **Model Evaluation**
   - Loads PneumoNet model from checkpoint
   - Uses standardized test data loader
   - No augmentation during evaluation

2. **Performance Metrics**
   - Binary classification metrics
   - ROC and PR curves analysis
   - Detailed per-class statistics

3. **Results Management**
   - JSON output with all metrics
   - Model information tracking
   - Checkpoint metadata storage

### Key Metrics
- Precision & Recall
- F1-score
- Accuracy
- ROC-AUC score
- PR-AUC
- Average Precision
- Per-class metrics

### Parameters
- `model_path`: Path to model checkpoint
- `data_dir`: Dataset location
- `results_dir`: Output directory
- `plot_cm`: Confusion matrix plotting flag

### Output
- Console summary of results
- Detailed JSON metrics file
- Model evaluation statistics

 ---

# Hyperparameter Tuning

## hyperparameter_tuning.py

### Key Components
def grid_search(data_dir):

Performs exhaustive grid search over specified hyperparameter ranges.


### Features
1. **Grid Search**
   - Exhaustive search over parameter combinations
   - Tracks best performing parameters
   - Stores all trial results

2. **Parameters Tuned**
   - Learning rate
   - Batch size
   - Number of epochs
   - Early stopping patience
   - Learning rate scheduler settings

3. **Results Tracking**
   - Best validation accuracy
   - Parameter combination history
   - Performance metrics per trial

### Returns
- Best parameters found
- Complete results history
- Validation accuracy for each combination

### Usage
Used to optimize model training parameters before final training run.

---

# Kaggle Dataset Downloader

## kaggle_downloader.py
Handles automated download and extraction of the chest X-ray pneumonia dataset from Kaggle.

### Key Components

def download_dataset_if_needed():

Downloads and organizes the chest X-ray dataset:
- Downloads from Kaggle if not present
- Extracts ZIP to downloads folder
- Moves train/val/test to data folder
- Cleans up temporary files

### Features
1. **Dataset Management**
   - Automatic Kaggle dataset download
   - Handles ZIP extraction
   - Organizes into train/val/test splits

2. **Directory Structure**
   - Creates necessary folders
   - Maintains clean project structure
   - Removes temporary files

3. **Error Handling**
   - Checks for existing data
   - Validates downloaded files
   - Ensures proper cleanup

### Requirements
- Kaggle API credentials
- `kaggle` Python package
- Sufficient disk space

### Output
Organized dataset in `data/` with:
- train/
- val/
- test/

---

# Model Architecture

## model.py
Defines the PneumoNet model architecture using a pre-trained ResNet18 backbone for pneumonia detection.

### Key Components

class PneumoNet(nn.Module):

Neural network for pneumonia detection:
- Based on ResNet18 architecture
- Configurable for binary/multiclass
- Optional pre-trained weights


def init(self, num_classes=1, use_pretrained=True):
- Initialize with ImageNet weights or from scratch
- Modifies final layer for classification task


### Features
1. **Model Architecture**
   - ResNet18 backbone
   - Modified final layer for classification
   - ImageNet pre-trained weights option

2. **Configuration Options**
   - Binary classification (num_classes=1)
   - Multiclass support (num_classes>1)
   - Flexible pre-training toggle

3. **Implementation Details**
   - Inherits from nn.Module
   - Uses modern weights parameter
   - Maintains ResNet architecture benefits

### Parameters
- `num_classes`: Output classes (1 for binary)
- `use_pretrained`: Toggle ImageNet weights

### Usage
Core model used in both training and inference pipelines

---

# Dataset Reorganization

## reorganize_dataset.py
Handles dataset reorganization for both binary(Normal/Pneumonia) and multiclass(Normal/Bacterial/Viral) classification.

### Key Components

def reorganize_dataset(data_dir="data", val_size=0.15, test_size=0.15):

Reorganizes chest X-ray dataset into two formats:
- Binary: data/ (NORMAL vs PNEUMONIA)
- Multiclass: data_multi/ (NORMAL vs BACTERIA vs VIRUS)

### Features
1. **Data Organization**
   - Creates binary classification structure
   - Creates multiclass classification structure
   - Maintains data splits (train/val/test)

2. **File Management**
   - Extracts pneumonia type from filenames
   - Handles file copying and moving
   - Maintains data integrity

3. **Split Configuration**
   - Configurable validation split
   - Configurable test split
   - Random shuffling of data


### Statistics
- Prints dataset distribution
- Shows class balance information
- Reports total images per category

---

# Multiclass Training

## train_multiclass.py
Implements training pipeline for three-class pneumonia classification (Normal vs. Bacterial vs. Viral).

### Key Components

class PneumoNetMulti(nn.Module):

Multi-class variant of PneumoNet:
- ResNet18 backbone
- 3-class output layer
- ImageNet pre-training option

def train_model_multiclass(data_dir, save_dir=None, results_dir=None):

Main training function with:
- Automatic mixed precision
- Early stopping
- Model checkpointing


### Features
1. **Training Pipeline**
   - Mixed precision training
   - Early stopping mechanism
   - Learning rate scheduling
   - Model checkpointing

2. **Hyperparameters**
   - Epochs: 50
   - Learning rate: 0.001
   - Batch size: 32
   - Early stopping patience: 10
   - Data augmentation options

3. **Metrics Tracking**
   - Loss (train/val)
   - Accuracy (train/val)
   - F1-score (train/val)
   - Precision/Recall
   - Confusion matrices

### Output
- Best model checkpoint
- Training history
- Performance metrics
- Confusion matrices

### Returns
- Trained model
- Best checkpoint path
- Training logs path

---

# Binary Training

## train.py
Implements the main training pipeline for binary pneumonia classification (Normal vs. Pneumonia).

### Key Components

def train_model(data_dir, save_dir=None, results_dir=None):

Main training function with:
- Automatic mixed precision (AMP)
- Early stopping
- Comprehensive metrics tracking


### Features
1. **Training Pipeline**
   - GPU/CPU support with AMP
   - Early stopping
   - Model checkpointing
   - Comprehensive logging

2. **Key Functions**
   - `train_epoch`: Single epoch training/validation
   - `train_model`: Complete training pipeline
   - Metrics calculation and logging

3. **Metrics Tracked**
   - Loss (train/val)
   - Precision/Recall
   - F1-score
   - Accuracy
   - AUC-ROC
   - Confusion matrices

### Output
- Model checkpoints
- Training history (JSON)
- Performance metrics
- Confusion matrices

### Returns
- Trained model
- Best model path
- Training logs path

---

# Utilities 

## utils.py
Contains utility functions and constants used in the project.


### Features
1. **Constants**
   - Class names for binary classification
   - Class names for multiclass classification
   - Matches filesystem organization

2. **GPU Utilities**
   - GPU availability check
   - Memory usage tracking
   - Device allocation info

3. **Checkpoint Management**
   - Model state saving
   - Optimizer state saving
   - Training metrics storage
   - Directory creation handling

### Usage
- Import class names for consistency
- Monitor GPU resources
- Save training progress
- Load model checkpoints

### Returns
- GPU statistics to console
- Checkpoint paths when saving

---

# **Notebooks**


# Model Analysis

## Analysis_of_models.ipynb
Jupyter notebook for analyzing and visualizing model performance after training.

### Key Components
**Main analysis sections**:
- Plot training/validation metrics
- Visualize model predictions
- Generate Grad-CAM visualizations


### Features
1. **Performance Visualization**
   - Training/validation loss curves
   - Accuracy, precision, recall plots
   - F1-score tracking
   - AUC-ROC curves

2. **Confusion Matrices**
   - Last three epochs analysis
   - Side-by-side train/val comparison
   - Class-wise performance

3. **Model Interpretation**
   - Grad-CAM heatmaps
   - Correct vs incorrect predictions
   - Attention visualization

### Key Plots
- Loss curves
- Metric curves (5 metrics)
- Confusion matrices
- Grad-CAM visualizations

### Dependencies
- torch
- matplotlib
- numpy
- PIL
- torchvision

### Usage
Run cells sequentially to:
1. Load training logs
2. Generate performance plots
3. Analyze model behavior
4. Visualize model attention

---

# Data Exploration

## data_exploration.ipynb

### Key Components

**Main exploration sections**:
- Dataset distribution analysis
- Image visualization
- Quality checks

### Features
1. **Dataset Statistics**
   - Distribution by class (normal/pneumonia)
   - Distribution by type (bacterial/viral)
   - Split ratios (train/val/test)
   - Class balance analysis

2. **Image Analysis**
   - Sample image visualization
   - Image size distribution
   - Grayscale vs color check
   - Brightness/contrast analysis

3. **Quality Checks**
   - Small file detection
   - Corrupted image check
   - Anomalous brightness detection
   - Image statistics (mean/std)

### Key Visualizations
- Bar charts of class distribution
- Sample images from each category
- Brightness distribution plots
- Image size histograms

### Dependencies
- pandas
- matplotlib
- PIL
- numpy

### Key Findings
- Dataset imbalance details
- Image quality metrics
- Potential preprocessing needs
- Anomaly detection results

---